In [30]:
cd ~/imCPA

/mnt/home/icb/alessandro.palma/imCPA


In [31]:
# Import the files 
import tarfile
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
from PIL import Image
import timeit,time
from plot_utils import Plotter 
import torch
from training_utils import *
import json
import sys

In [32]:
torch.cuda.set_per_process_memory_fraction(1.0, device=None)

In [33]:
config = Config(config_path = '/home/icb/alessandro.palma/imCPA/configs/config_sigma_vae.json') 

In [34]:
t = Trainer(config)

Working on device: cuda
Lodading the data...
Successfully loaded the data


In [ ]:
t.train()

In [ ]:
model = t.model.module
checkpoint = torch.load('/home/icb/alessandro.palma/checkpoints/vae_run_res_100')
model.load_state_dict(checkpoint['model_state_dict'])

In [7]:
data = next(iter(t.loader_train))

In [9]:
data.keys()

dict_keys(['X', 'file_name', 'mol_name', 'mol_one_hot', 'assay_labels', 'state', 'drug_embedding'])

In [28]:
np.argmax(data['mol_one_hot'],  1)

tensor([ 6168,  5245,  9216,  4171,   557,  7284,  8059, 10144,  5976,  6447,
         6149,  9310,  7424,  3761,  5256,  8870,  4332,  5973,  7354, 10171,
         9865,  2063,   919,  9715,  4936, 10160,  1164,  2535,  9571,  9509,
         6767,  3688,  7078,  4581,  6255, 10260,  9308,     1,  4821,  2439,
         7255,  4887,  3905,  9744,  5123,  3002,  8438,  9270,  9411,  9550,
         4097,  9023,  6856,  8996,  4649,  5165,  3717,  5786,  9464,  1073,
         1767,  9777,  7230,  6677,  4067,  5095,  9771, 10118,  9515,  8356,
         4914,  8126,  9029,  9391,  1103,   391,  9023,  1619,  2820,  8929,
         3890,  2371,  4276, 10068,  9510,  6226,  7997,   631,  1137,  7242,
         4117,   125,  1514,  7992,  1726,  1705,  6828,  8124,  1502,  2298,
         5312,  6061,  9023,  5207,  8840, 10069,  8720,  6989,  9747,  1758,
         5456,  7848,  9814,  4907,  1250,   695,  9013,  9244,  7719,  5386,
         8941,  5800,  8442,  9715,  2207,  2946,  9462,  4314])

In [30]:
print(data['mol_name'])

['BRD-K61105570', 'BRD-K48612418', 'RS 39604', 'BRD-K34029905', 'BRD-A24067767', 'BRD-K76144394', 'BRD-K86845405', 'sulfapyridine', 'BRD-K58498516', 'BRD-K64544620', 'BRD-K60894184', 'Y27632', 'BRD-K78142829', 'BRD-K28134384', 'BRD-K48788686', 'BRD-K98303556', 'BRD-K36196444', 'BRD-K58486677', 'BRD-K77162186', 'thiamphenicol', 'methapyrilene', 'BRD-K05299321', 'BRD-A50048201', 'fusaric acid', 'BRD-K44464569', 'telenzepine', 'BRD-A64849281', 'BRD-K11237957', 'diazepam', 'cinoxacin', 'BRD-K68756555', 'BRD-K27279217', 'BRD-K73319967', 'BRD-K39597125', 'BRD-K62191543', 'zopiclone', 'XE 991', '(+)-UH 232', 'BRD-K43011399', 'BRD-K09944219', 'BRD-K75777117', 'BRD-K43873821', 'BRD-K30063533', 'hesperidin', 'BRD-K47001216', 'BRD-K17606413', 'BRD-K92014291', 'SKI-II', 'benzo(a)pyrene', 'cytisine', 'BRD-K32818886', 'DMSO', 'BRD-K70021293', 'CCMQ', 'BRD-K40472535', 'BRD-K47615243', 'BRD-K27618188', 'BRD-K56209150', 'carbacyclin', 'BRD-A59790839', 'BRD-K01367395', 'indometacin', 'BRD-K75439260', 'B

In [29]:
data['assay_labels']

tensor([[ 0.,  0.,  0.,  ...,  0.,  0., -1.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]], dtype=torch.float64)

In [ ]:
z = model(data['X'].to(t.device))['z']

In [65]:
torch.exp(-F.gaussian_nll_loss(torch.zeros(512, 512).to(t.device), z, 
                                torch.ones(512).to(t.device), eps=1e-06, reduction='mean'))

tensor(0., device='cuda:0', grad_fn=<ExpBackward>)